[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PennNGG/Quantitative-Neuroscience/blob/master/Concepts/Python/Multiple%20Comparisons.ipynb)

# Definitions

The multiple comparisons problem in statistics occurs when multiple statistical inferences are done simultaneously, which greatly increases the probability that any one inference will yield an erroneous result, by chance. A lot has been written about this problem, including:

- [Its prevalence in fMRI data analysis](https://www.sciencedirect.com/science/article/pii/S1053811912007057?via%3Dihub) (including a compelling illustration by this [prizewinning study](https://blogs.scientificamerican.com/scicurious-brain/ignobel-prize-in-neuroscience-the-dead-salmon-study/)\)

- [How Baysian methods can avoid the problem](http://www.stat.columbia.edu/~gelman/research/published/multiple2f.pdf).

- [General approaches for correcting for multiple comparisons](http://www.biostathandbook.com/multiplecomparisons.html).

Here we will provide some intuition for the problem using a simple thought experiment, to sensitize you to how much of a problem it can be. Consider performing the same statistical test on *N* different samples corresponding to, say, different voxels in fMRI data, using a *p*-value of $\alpha$ (typically 0.05) for each test. 

Thus, for any one test, the probability of getting a Type I error (rejecting $H_0$ when $H_0$ is true) is $\alpha$:

$p_{error}=\alpha$

For two tests, the probably of getting a Type I error for either test is just one minus the combined probability of not getting a Type I error from either one:

$p_{error}=1-(1-\alpha)(1-\alpha)$

For *N* tests, the probably of getting a Type I error for either test is just one minus the combined probability of not getting a Type I error from any one:

$p_{error}=1-(1-\alpha)^N$

Run the cell below to see that the probability of getting a Type I error under these conditions grows rapidly with *N*, implying that it becomes very, very likely that you will get a "statistically significant result" just by chance if you do enough tests:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

alpha = 0.05
N = np.arange(0,100)
plt.plot(N, 1-(1-alpha)**N)
plt.xlabel('N')
plt.ylabel('$P_{error}$')

# Correcting for multiple comparisons



There are a number of different methods that can be used to [correct for this problem](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5506159/). Below are two common methods.




## Bonferroni correction

The simplest way to correct for Type I errors (false positives) in multiple comparisons is to divide $\alpha$ by the number of comparisons, known as the [Bonferroni correction](https://mathworld.wolfram.com/BonferroniCorrection.html). This is a very conservative test that is typically used when the number of comparisons is relatively small and you want to avoid Type I errors.

## Benjamini–Hochberg procedure


Another approach is to more carefully control the false-discovery rate using the [Benjamini–Hochberg procedure](https://www.jstor.org/stable/2346101?seq=1#metadata_info_tab_contents):

1\. Rank the individual *p*-values in ascending order, labeled *i*=1...*n*

2\. For each *p*-value, calculate its "critical value" as (*i*/*n*)*Q*, where *i* is the rank, *n* is the total number of tests, and *Q* is the false discovery rate (a percentage) that you choose (typically 0.05).

3\. In your rank-ordered, original *p*-values, find the largest value that is smaller than its associated critical value; this *p*-value is the new criterion (i.e., reject $H_0$ for all cases for which *p* ≤ this value).

# Exercise

In this exercise we will run through an example of correcting for multiple comparisons with both the Benjamini-Hochberg procedure and the more conservative Bonferroni correction. 

First, simulate multiple (say, 1000) t-tests comparing two samples with equal means and standard deviations, and save the p-values. Obviously, at p<0.05 we expect that ~5% of the simulations to yield a "statistically significant" result (of rejecting the NULL hypothesis that the samples come from distributions with equal means).

Second, once you have the simulated p-values, apply both methods to address the multiple comparisons problem.

Third, set the sample 1 and sample 2 means to be 1 and 2 respectively, and re-run the exercise. What do you notice? What if you make the difference between means even greater?

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st

# Define unpaired measurements, same sigma, same mu
mu = 1
sigma = 1
sig_counter = 0

# 1000 simulations of t tests
for n in range(1000):
  # Get random samples, same n
  N = 10
  X1 = np.random.normal(mu, sigma, N)
  X2 = np.random.normal(mu, sigma, N)

  # Calculate stats using ttest (use ttest_ind for two independent samples)
  tstat, pval = st.ttest_ind(X1, X2)

  # Count sig p values
  if pval <= 0.05:
    sig_counter = sig_counter + 1

# Calculate percent of sig results out of 1000
percent_sig = (sig_counter/1000)*100
print(f'For two normal distributions with the same parameters, the proportion of "statistically significant" t-test results out of 1000 is {percent_sig:.2f}%.')

In [ ]:
# Define unpaired measurements, same sigma, same mu, 1000 simulations
mu = 1
sigma = 1
sig_counter = 0
sim = 1000

# N simulations of t-tests with Bonferroni 
for n in range(sim):
  # Get random samples, same n
  N = 10
  X1 = np.random.normal(mu, sigma, N)
  X2 = np.random.normal(mu, sigma, N)

  # Calculate stats using ttest (use ttest_ind for two independent samples)
  tstat, pval = st.ttest_ind(X1, X2)

  # Count sig p values with Bonferroni
  alpha = 0.05/sim
  if pval <= alpha:
    sig_counter = sig_counter + 1

# Calculate percent of sig results out of 1000
percent_sig = (sig_counter/sim)*100
print(f'For two normal distributions with the same parameters, the proportion of "statistically significant" t-test results with Bonferroni correction out of {sim} is {percent_sig:.2f}%.')

In [ ]:
# Define unpaired measurements, same sigma, same mu, 1000 simulations
mu = 1
sigma = 1
p_vals = []
sig_counter = 0
sim = 1000
Q = 0.05

# N simulations of t-tests
for n in range(sim):
  # Get random samples, same n
  N = 10
  X1 = np.random.normal(mu, sigma, N)
  X2 = np.random.normal(mu, sigma, N)

  # Calculate stats using ttest, add p val to list
  tstat, pval = st.ttest_ind(X1, X2)
  p_vals.append(pval)

# Sort p values from largest to smallest
p_vals.sort(reverse=True)

# Calculate critical value for each p value
for n in range(sim):
  crit_val = ((n+1)/sim)*Q
  # Detemrine largest p value that is smaller than associated critical value, set a new p value
  if p_vals[n] < crit_val:
    new_p = p_vals[n]
    break

print(f'The new p value determined by Benhamini_Hochberg procedure is {new_p:.4f}.')

# Count number of p values <= new p value
for n in range(sim):
  if p_vals[n] <= new_p:
    sig_counter = sig_counter + 1

# Calculate percent of sig results out of 1000
percent_sig = (sig_counter/sim)*100

print(f'For two normal distributions with the same parameters, the proportion of "statistically significant" t-test results out of {sim} is {percent_sig:.2f}%.')

In [ ]:
# Define unpaired measurements, same sigma, -different mu
mu_one = 1
mu_two = 2
sigma = 1
sim = 1000
sig_counter = 0

# 1000 simulations of t tests, no correction
for n in range(sim):
  # Get random samples, same n
  N = 10
  X1 = np.random.normal(mu_one, sigma, N)
  X2 = np.random.normal(mu_two, sigma, N)

  # Calculate stats using ttest (use ttest_ind for two independent samples)
  tstat, pval = st.ttest_ind(X1, X2)

  # Count sig p values
  if pval <= 0.05:
    sig_counter = sig_counter + 1

# Calculate percent of sig results out of 1000
percent_sig = (sig_counter/sim)*100
print(f'For two normal distributions with a mean of 1 and 2 and a shared sigma of 1, the proportion of "statistically significant" t-test results out of {sim} is {percent_sig:.2f}%.\n')



# N simulations of t-tests with Bonferroni 
sig_counter_b = 0

for n in range(sim):
  # Get random samples, same n
  N = 10
  X1 = np.random.normal(mu_one, sigma, N)
  X2 = np.random.normal(mu_two, sigma, N)

  # Calculate stats using ttest (use ttest_ind for two independent samples)
  tstat, pval = st.ttest_ind(X1, X2)

  # Count sig p values with Bonferroni
  alpha = 0.05/sim
  if pval <= alpha:
    sig_counter_b = sig_counter_b + 1

# Calculate percent of sig results out of 1000
percent_sig = (sig_counter_b/sim)*100
print(f'The proportion of "statistically significant" t-test results with Bonferroni correction out of {sim} is {percent_sig:.2f}%.\n')



# Calculate with Benhamini_Hochberg
p_vals = []
sig_counter_BH = 0
Q = 0.05

# N simulations of t-tests
for n in range(sim):
  # Get random samples, same n
  N = 10
  X1 = np.random.normal(mu_one, sigma, N)
  X2 = np.random.normal(mu_two, sigma, N)

  # Calculate stats using ttest, add p val to list
  tstat, pval = st.ttest_ind(X1, X2)
  p_vals.append(pval)

# Sort p values from largest to smallest
p_vals.sort(reverse=True)

# Calculate critical value for each p value
for n in range(sim):
  crit_val = ((n+1)/sim)*Q
  # Detemrine largest p value that is smaller than associated critical value, set a new p value
  if p_vals[n] < crit_val:
    new_p = p_vals[n]
    break

print(f'The new p value determined with Benhamini_Hochberg procedure is {new_p:.4f}.')

# Count number of p values <= new p value
for n in range(sim):
  if p_vals[n] <= new_p:
    sig_counter_BH = sig_counter_BH + 1

# Calculate percent of sig results out of 1000
percent_sig = (sig_counter_BH/sim)*100

print(f'The proportion of "statistically significant" t-test results out of {sim} is {percent_sig:.2f}%.')

# Additional Resources


How to correct for multiple comparisons in [Matlab](https://www.mathworks.com/help/stats/multcompare.html), [R](https://stat.ethz.ch/R-manual/R-devel/library/stats/html/p.adjust.html), and [Python](https://www.statsmodels.org/dev/generated/statsmodels.stats.multitest.multipletests.html)

# Credits

Copyright 2021 by Joshua I. Gold, University of Pennsylvania